In [2]:
import networkx as nx
import gensim.models
from os import walk
import numpy as np
import pandas as pd
import pickle
import sys
import re
import semanticanalysis as sa
import functools
import itertools
from multiprocessing import Pool
import sys
import pickle

## Single Network

In [34]:
L = 5
G = nx.read_gexf('results/srcnets/trump-999.gexf')
nodes = list(sorted(G.nodes()))
print('\\begin{tabular}{ l |', ' '.join(['c',]*len(G.nodes())), '}')
print('\\hline')
print('sources &', ' & '.join([v.split('_')[0] for v in nodes]), '\\\\')
print('\\hline')
for u in nodes:
    srcname = u.split('_')[0]
    weights = [(v, G.edge[u][v]['weight']) for v in nodes]
    wstrings = ['{0:.3f}'.format(w) if v < u else '-' for v,w in weights]
    print(srcname, '&', ' & '.join(wstrings), '\\\\')
    
print('\\end{tabular}')

\begin{tabular}{ l | c c c c c c c c c c }
\hline
sources & breitbart & cbsnews & cnn & foxnews & huffpo & nytimes & reuters & usatoday & wapo & watimes \\
\hline
breitbart & - & - & - & - & - & - & - & - & - & - \\
cbsnews & 0.505 & - & - & - & - & - & - & - & - & - \\
cnn & 0.597 & 0.506 & - & - & - & - & - & - & - & - \\
foxnews & 0.612 & 0.597 & 0.730 & - & - & - & - & - & - & - \\
huffpo & 0.400 & 0.332 & 0.542 & 0.500 & - & - & - & - & - & - \\
nytimes & 0.494 & 0.409 & 0.652 & 0.608 & 0.665 & - & - & - & - & - \\
reuters & 0.426 & 0.346 & 0.551 & 0.510 & 0.540 & 0.603 & - & - & - & - \\
usatoday & 0.399 & 0.544 & 0.359 & 0.416 & 0.291 & 0.332 & 0.278 & - & - & - \\
wapo & 0.481 & 0.380 & 0.639 & 0.572 & 0.672 & 0.731 & 0.597 & 0.303 & - & - \\
watimes & 0.506 & 0.417 & 0.641 & 0.610 & 0.629 & 0.711 & 0.579 & 0.364 & 0.723 & - \\
\end{tabular}


## Network Comparison

In [36]:
L = 5
G = nx.read_gexf('results/srcnets/military-800.gexf')
H = nx.read_gexf('results/srcnets/trump-800.gexf')
nodes = list(sorted(G.nodes()))
print('\\begin{tabular}{ l |', ' '.join(['c',]*len(G.nodes())), '}')
print('\\hline')
print('sources &', ' & '.join([v.split('_')[0] for v in nodes])+'}', '\\\\')
print('\\hline')
for u in nodes:
    srcname = u.split('_')[0]
    weights = [(v, G.edge[u][v]['weight']-H.edge[u][v]['weight']) for v in nodes]
    wstrings = ['{0:.3f}'.format(w) if v < u else '-' for v,w in weights]
    print(srcname, '&', ' & '.join(wstrings), '\\\\')
    
print('\\end{tabular}')

\begin{tabular}{ l | c c c c c c c c c c }
\hline
sources & breitbart & cbsnews & cnn & foxnews & huffpo & nytimes & reuters & usatoday & wapo & watimes} \\
\hline
breitbart & - & - & - & - & - & - & - & - & - & - \\
cbsnews & -0.026 & - & - & - & - & - & - & - & - & - \\
cnn & -0.010 & -0.114 & - & - & - & - & - & - & - & - \\
foxnews & -0.129 & -0.118 & -0.092 & - & - & - & - & - & - & - \\
huffpo & 0.040 & -0.061 & 0.099 & 0.051 & - & - & - & - & - & - \\
nytimes & -0.143 & -0.175 & -0.081 & -0.094 & 0.013 & - & - & - & - & - \\
reuters & -0.062 & -0.149 & -0.112 & -0.154 & -0.086 & -0.144 & - & - & - & - \\
usatoday & -0.007 & 0.152 & -0.037 & -0.080 & -0.115 & -0.174 & -0.135 & - & - & - \\
wapo & -0.082 & -0.130 & -0.022 & -0.063 & 0.033 & -0.046 & -0.112 & -0.142 & - & - \\
watimes & 0.030 & -0.059 & 0.032 & -0.015 & 0.034 & -0.096 & -0.064 & -0.122 & -0.051 & - \\
\end{tabular}
